In [1]:
# import dependancies
import pandas as pd
import random
from datetime import datetime, timedelta
from yahoo_fin.stock_info import *
import yfinance as yf
import time
from selenium import webdriver
from bs4 import BeautifulSoup
from splinter import Browser
from pprint import pprint
import finnhub
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

# change to your api keys 

#alphavantage_api_key = 'AJP4EKR4LE52NNK4'
alphavantage_api_key = 'CORYDT1RMNVL3DVT'
polygon_api_key = 'J9UUfkyTylf3D_s76b5pcMCnLSaKTNpD'
finnhub_client = finnhub.Client(api_key="cld35g9r01qn959jif1gcld35g9r01qn959jif20")


### Screening US Mega & Large Cap Stocks

In [2]:
file_path = 'CSV files/nasdaq_screener_1700335987448.csv'

# Read the CSV file into a Pandas DataFrame
stock_list_df = pd.read_csv(file_path)

# Display the DataFrame
stock_list_df.head()

,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry
0,A,Agilent Technologies Inc. Common Stock,$113.15,-1.04,-0.91%,3.310625e+10,United States,1999.0,2700770,Industrials,Electrical Products
1,AAPL,Apple Inc. Common Stock,$189.69,-0.02,-0.01%,2.950200e+12,United States,1980.0,50922657,Technology,Computer Manufacturing
2,ABBV,AbbVie Inc. Common Stock,$138.30,0.02,0.01%,2.441740e+11,United States,2012.0,4332297,Health Care,Biotechnology: Pharmaceutical Preparations
3,ABNB,Airbnb Inc. Class A Common Stock,$127.15,0.87,0.69%,8.244890e+10,United States,2020.0,3795704,Consumer Discretionary,Hotels/Resorts
4,ABT,Abbott Laboratories Common Stock,$99.55,-0.71,-0.71%,1.728250e+11,United States,NaN,6069070,Health Care,Biotechnology: Pharmaceutical Preparations


In [3]:
# create an array of tickers

ticker_list = stock_list_df['Symbol'].values

### Generating functions to retrieve stock information 

In [4]:
# define function to generate random date, and ensure that the date is a weekday, maybe start in 2014 or 2015

def generate_random_date(start_date, end_date):
    check_weekday = False
    while not check_weekday:
        # Convert the date strings to datetime objects
        start_date_dt = datetime.datetime.strptime(start_date, '%Y-%m-%d')
        end_date_dt = datetime.datetime.now()  # Using today's date as the end date

        # Calculate the range of days between the start and end date
        delta_days = (end_date_dt - start_date_dt).days

        # Generate a random number of days within the range
        random_days = random.randint(0, delta_days)

        # Calculate the random date
        random_date = start_date_dt + timedelta(days=random_days)

        # Check if the generated date is a weekday
        if random_date.weekday() < 5:
            check_weekday = True
            return random_date
        else:
            check_weekday = False



In [5]:
# define a function to choose a random stock from the list

def choose_random_stock(ticker_list):    
    
    # Randomly choose a stock from the list
    random_stock = random.choice(ticker_list)
    
    return random_stock.lower()
    

In [6]:
# define a function to check if data exists for stock on the date 

def check_stock_date(ticker, check_date):
    check_financials = False
    # Check if the input date is a Friday
    if check_date.weekday() == 4:  # Friday
        # Set end date to the next Monday
        adj_end_date = check_date + timedelta(days=3)
    else:
        # Set end date to the next day
        adj_end_date = check_date + timedelta(days=1)
    
    # using yahoo finance function to get stock data dataframe
    
    
    try:
        get_data(ticker, end_date=adj_end_date)
        get_stock_financials(ticker, check_date)
        

        check_financials = True
    except:
        check_financials = False
    
    if not check_financials:
        return False
    else:
        return True
    

In [7]:
# define a function to return stock industry

def return_stock_industry(symbol):
    symbol = symbol.upper()
    # Check if the symbol is in the DataFrame
    if symbol in stock_list_df['Symbol'].values:
        # Retrieve the sector for the given symbol
        sector = stock_list_df.loc[stock_list_df['Symbol'] == symbol, 'Sector'].iloc[0]
        return sector
    else:
        return f"Symbol '{symbol}' not found in the stock list."

In [8]:

# define a function to get the adjusted closing price of a stock on a particular day

def get_stock_price(ticker, stock_date):
    # Check if the input date is a Friday
    if stock_date.weekday() == 4:  # Friday
        # Set end date to the next Monday
        price_adj_end_date = stock_date + timedelta(days=3)
    else:
        # Set end date to the next day
        price_adj_end_date = stock_date + timedelta(days=1)
    
    # using yahoo finance function to get stock data dataframe
    stock_data = get_data(ticker, end_date=price_adj_end_date)
    # extracting the price 
    adj_close_price = stock_data['adjclose'].values[-1]
    
    return adj_close_price

In [9]:
# define a function to get stock trading volume

def get_stock_volume(ticker, vol_date):
    # Check if the input date is a Friday
    if vol_date.weekday() == 4:  # Friday
        # Set end date to the next Monday
        vol_adj_end_date = vol_date + timedelta(days=3)
    else:
        # Set end date to the next day
        vol_adj_end_date = vol_date + timedelta(days=1)
    
    # using yahoo finance function to get stock data dataframe
    stock_data = get_data(ticker, end_date=vol_adj_end_date)
    
    #extracting the volume
    trading_vol = stock_data['volume'].values[-1]
    
    return trading_vol

In [10]:
# function to get 4 previous weekdays from given_date

def get_previous_weekdays(weekday_date):
    previous_weekdays = []
    previous_date = weekday_date - timedelta(days=1)
    while len(previous_weekdays) < 4:
        if previous_date.weekday() < 5:
            previous_weekdays.append(previous_date)
        previous_date -= timedelta(days=1)

    return previous_weekdays


In [11]:
# function to get company's financial ratios by scraping stockanalysis.com

def get_stock_financials(ticker, fin_date):
    
    financial_ratios = {}

    # setting brwoser
    browser = Browser('chrome')
    
    # URL of the website
    url = f"https://stockanalysis.com/stocks/{ticker}/financials/ratios/?p=quarterly"

    # Use Selenium to open the webpage and let JavaScript execute
    driver = webdriver.Chrome()  # Make sure you have the ChromeDriver executable in your PATH
    driver.get(url)

    # Allow some time for JavaScript to execute and load the content
    time.sleep(5)  # Adjust the sleep time based on your needs

    # Get the page source after JavaScript execution
    page_source = driver.page_source

    # Close the Selenium WebDriver
    driver.quit()

    # Parse the HTML content of the page
    soup = BeautifulSoup(page_source, "html.parser")

    table = soup.find("table", class_="w-full")

    # Check if the table is found
    try:
        # Use pandas to read the HTML table into a DataFrame
        df = pd.read_html(str(table))[0]
        # Extract date columns excluding the first column
        date_columns = df.columns[2:-1]
        
        # Convert date columns to datetime objects
        date_columns = pd.to_datetime(date_columns)
        
        # Find the closest date column to the given date that is earlier
        earlier_dates = [date for date in date_columns if date < fin_date]
        closest_date_column = max(earlier_dates)

        # Access the closest column
        closest_column = closest_date_column.strftime('%Y-%m-%d')
        
    except:
        print("Table not found on the page.")
    
    

    #extracting financial ratios
    try: 
        financial_ratios['pe_ratio']=float(df.loc[df['Quarter Ended'] == 'PE Ratio', closest_column].values[0])
        financial_ratios['debt_to_equity']=float(df.loc[df['Quarter Ended'] == 'Debt / Equity Ratio', closest_column].values[0])
        financial_ratios['quick_ratio']=float(df.loc[df['Quarter Ended'] == 'Quick Ratio', closest_column].values[0])
        total_shareholder_return_str = df.loc[df['Quarter Ended'] == 'Total Shareholder Return', closest_column].values[0]
        financial_ratios['total_shareholder_return'] = float(total_shareholder_return_str.strip('%'))
    
    except:
        pass 
    
    # URL of the website
    url = f"https://stockanalysis.com/stocks/{ticker}/financials/?p=quarterly"

    # Use Selenium to open the webpage and let JavaScript execute
    driver = webdriver.Chrome()  # Make sure you have the ChromeDriver executable in your PATH
    driver.get(url)

    # Allow some time for JavaScript to execute and load the content
    time.sleep(5)  # Adjust the sleep time based on your needs

    # Get the page source after JavaScript execution
    page_source = driver.page_source

    # Close the Selenium WebDriver
    driver.quit()

    # Parse the HTML content of the page
    soup = BeautifulSoup(page_source, "html.parser")

    # Find the table by class name
    table = soup.find("table", class_="w-full")

    # Check if the table is found
    try:
    # Use pandas to read the HTML table into a DataFrame
        df = pd.read_html(str(table))[0]
        
    except:
        print("Table not found on the page.")
    
    try:
        profit_margin_str= df.loc[df['Quarter Ended'] == 'Profit Margin', closest_column].values[0]
        financial_ratios['profit_margin'] = float(profit_margin_str.strip('%'))
        cash_margin_str= df.loc[df['Quarter Ended'] == 'Free Cash Flow Margin', closest_column].values[0]
        financial_ratios['free_cash_margin'] = float(cash_margin_str.strip('%'))
    except:
        pass

    return financial_ratios
    
    

In [12]:
# function to calculate stock's volatility

def calculate_volatility(ticker, sd_date):
    df=get_data(ticker, end_date=sd_date)
    # Calculate daily returns
    df['daily_return'] = df['adjclose'].pct_change()

    # Calculate volatility as the standard deviation of daily returns
    volatility = df['daily_return'].std()

    return volatility

In [13]:
#------------------------NEED TO FIND A NEW SOURCE---------------------------------

def get_news_sentiment(ticker, news_date):
    
    # Extract year, month, and day
    news_end_date=news_date
    end_year = news_end_date.year
    end_month = str(news_end_date.month).zfill(2)
    end_day = str(news_end_date.day).zfill(2)
    
    news_start_date = news_end_date - timedelta(weeks=4)
    start_year = news_start_date.year
    start_month = str(news_start_date.month).zfill(2)
    start_day = str(news_start_date.day).zfill(2)
   

In [14]:
# extracting daily federal interest rate information from alphavantage

# using alphavantage API
url = f'https://www.alphavantage.co/query?function=FEDERAL_FUNDS_RATE&interval=daily&apikey={alphavantage_api_key}'
r = requests.get(url)
data = r.json()

daily_interest_rates = data

# converting json to dataframe
daily_interest_rate_df = pd.DataFrame(daily_interest_rates['data'])

# Export DataFrame to CSV so that we can refer to it without making an api call
daily_interest_rate_df.to_csv('daily_interest_rates.csv', index=False)


In [15]:
# extracting monthly CPI information from alphavantage

# using alphavantage api
url = f'https://www.alphavantage.co/query?function=CPI&interval=monthly&apikey=demo'
r = requests.get(url)
data = r.json()

monthly_cpi = data

# converting json to dataframe
monthly_cpi_df = pd.DataFrame(monthly_cpi['data'])
monthly_cpi_df['date'] = pd.to_datetime(monthly_cpi_df['date'])


# Export DataFrame to CSV so that we can refer to it without making an api call
monthly_cpi_df.to_csv('monthly_cpi.csv', index=False)

In [16]:
# extracting monthly unemployment rate information from alphavantage

# using alphavantage api
url = f'https://www.alphavantage.co/query?function=UNEMPLOYMENT&apikey=demo'
r = requests.get(url)
data = r.json()

monthly_unemployment_rate = data

# converting json to dataframe
monthly_unemployment_rate_df = pd.DataFrame(monthly_unemployment_rate['data'])
monthly_unemployment_rate_df['date'] = pd.to_datetime(monthly_unemployment_rate_df['date'])

# Export DataFrame to CSV so that we can refer to it without making an api call
monthly_unemployment_rate_df.to_csv('monthly_unemployment_rate.csv', index=False)

In [17]:
# define a function to extract economic indicators for the given date

def get_economic_indicators(econ_date):
    
    economic_indicators_dict={}
    
    # convert datetime object to matching string 
    formatted_date = econ_date.strftime('%Y-%m-%d')
    
    # retrieve CPI value
    # Locate the value for the given date, matching only month and year
    economic_indicators_dict['cpi'] = float(monthly_cpi_df.loc[monthly_cpi_df['date'].dt.to_period('M') == pd.to_datetime(econ_date).to_period('M'), 'value'].iloc[0])    
    
    # retrieve federal interest rate value
    economic_indicators_dict['interest_rate'] = float(daily_interest_rate_df.loc[daily_interest_rate_df['date'] == formatted_date, 'value'].iloc[0])

    # retrieve unemployement rate
    # Locate the value for the given date, matching only month and year
    economic_indicators_dict['unemployment_rate'] = float(monthly_unemployment_rate_df.loc[monthly_unemployment_rate_df['date'].dt.to_period('M') == pd.to_datetime(econ_date).to_period('M'), 'value'].iloc[0])
    
    return economic_indicators_dict

In [18]:
# function to retrieve technical indicators

def get_technical_indicators(ticker, tech_date):
    
    technical_indicators_dict={}
    
    # changing date into UNIX format
    end_timestamp = int(tech_date.timestamp())
    
    # choosing a timestamp of one week earlier
    start_tech_date = tech_date - timedelta(weeks=1)
    
    # changing the date to UNIX format
    start_timestamp = int(start_tech_date.timestamp())
    
    # retrieving SMA 
    technical_indicators_dict['sma']= finnhub_client.technical_indicator(symbol=ticker, resolution='D', _from=start_timestamp, to=end_timestamp, indicator='sma', indicator_fields={"timeperiod": 5})['sma'][-1]
    
    #retrieving RSI
    technical_indicators_dict['rsi']= finnhub_client.technical_indicator(symbol=ticker, resolution='D', _from=start_timestamp, to=end_timestamp, indicator='rsi', indicator_fields={"timeperiod": 5})['rsi'][-1]

    # retrieving EMA 
    technical_indicators_dict['ema']= finnhub_client.technical_indicator(symbol=ticker, resolution='D', _from=start_timestamp, to=end_timestamp, indicator='ema', indicator_fields={"timeperiod": 5})['ema'][-1]
    
    return technical_indicators_dict

In [19]:
# function to create label value for data

def create_label(ticker, y_date):
    print(y_date)
    initial_price = get_stock_price(ticker, y_date)
    print(initial_price)
    
    # find the next weekday
    
    # check if date landed on friday and calculate the next weekday date
    if y_date.weekday() == 4:
        next_weekday_date = y_date + timedelta(days=3)
    else:
        next_weekday_date = y_date + timedelta(days=1)

    print(next_weekday_date)
    
    next_price = get_stock_price(ticker, next_weekday_date)
    
    print(next_price)
    
    if next_price >= initial_price:
        return 1 # 1 is class increase, in order to simplify the classification we put stayed the same in same category as increase
    else:
        return 0 # 0 is class decrease

### Running a loop to generate multiple rows of data for training and testing

In [20]:
page=10

In [22]:
# list to store dictionaries
list_dict = []
page += 1
# runs loop until list_dic [change no. 4 as desired]
while len(list_dict) < 500:

    # data_dict
    data_dict = {}
    
    # checking to see if generated ticker-date combo has available data
    checker = False
    
    # runs this loop until checker is true
    while not checker:

        ticker = choose_random_stock(ticker_list)
        input_date = generate_random_date('2017-01-01', '2023-10-31')

        checker = check_stock_date(ticker, input_date)

    # using function to return stock industry
    data_dict['industry']=return_stock_industry(ticker)

    # retrieve weekly returns
    date = input_date
    for i in range(1, 5):
        data_dict[f'wr{i}']=(get_stock_price(ticker, date) - get_stock_price(ticker, date - timedelta(weeks = 1)))/get_stock_price(ticker, date - timedelta(weeks = 1))
        date = date - timedelta(weeks=1)

    # retrieve last 5 working day volumes
    data_dict['vol1'] = get_stock_volume(ticker, date)
    for i, weekday in enumerate(get_previous_weekdays(input_date)):
        data_dict[f'vol{i+1}'] = get_stock_volume(ticker, weekday)

    # retrieve stock financials    
    for key, value in get_stock_financials(ticker, input_date).items():
        data_dict[key] = value
    
    # retrieve stock volatility
    data_dict['volatility'] = calculate_volatility(ticker, input_date)

    # retrieve economic indicators
    for key, value in get_economic_indicators(input_date).items():
        data_dict[key] = value

    # retrieve stocktechnical indicators    
    for key, value in get_technical_indicators(ticker, input_date).items():
        data_dict[key] = value

    # generate a label for row    
    data_dict['label'] = create_label(ticker, input_date)
        
    # append dictionary to list
    list_dict.append(data_dict)
    print(len(list_dict))

2019-03-18 00:00:00
187.50999450683594
2019-03-19 00:00:00
191.52999877929688
1
2017-06-02 00:00:00
56.93358612060547
2017-06-05 00:00:00
56.71973419189453
2
2022-08-23 00:00:00
169.62913513183594
2022-08-24 00:00:00
170.4398651123047
3
2019-10-14 00:00:00
51.983272552490234
2019-10-15 00:00:00
52.62092208862305
4
2022-01-10 00:00:00
19.617347717285156
2022-01-11 00:00:00
19.727943420410156
5
2023-06-08 00:00:00
122.13999938964844
2023-06-09 00:00:00
122.2300033569336
6
2022-05-03 00:00:00
98.14420318603516
2022-05-04 00:00:00
100.73320007324219
7
2020-05-05 00:00:00
207.65536499023438
2020-05-06 00:00:00
206.09986877441406
8
2021-06-24 00:00:00
817.7998046875
2021-06-25 00:00:00
824.1077270507812
9
2020-02-17 00:00:00
152.41000366210938
2020-02-18 00:00:00
152.66000366210938
10
2018-04-12 00:00:00
88.93524169921875
2018-04-13 00:00:00
87.16064453125
11
2019-09-05 00:00:00
145.68223571777344
2019-09-06 00:00:00
145.25497436523438
12
2021-02-26 00:00:00
41.56505584716797
2021-03-01 00:0

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=119.0.6045.160)
Stacktrace:
	GetHandleVerifier [0x00007FF717A982B2+55298]
	(No symbol) [0x00007FF717A05E02]
	(No symbol) [0x00007FF7178C05AB]
	(No symbol) [0x00007FF7178A0038]
	(No symbol) [0x00007FF717926BC7]
	(No symbol) [0x00007FF71793A15F]
	(No symbol) [0x00007FF717921E83]
	(No symbol) [0x00007FF7178F670A]
	(No symbol) [0x00007FF7178F7964]
	GetHandleVerifier [0x00007FF717E10AAB+3694587]
	GetHandleVerifier [0x00007FF717E6728E+4048862]
	GetHandleVerifier [0x00007FF717E5F173+4015811]
	GetHandleVerifier [0x00007FF717B347D6+695590]
	(No symbol) [0x00007FF717A10CE8]
	(No symbol) [0x00007FF717A0CF34]
	(No symbol) [0x00007FF717A0D062]
	(No symbol) [0x00007FF7179FD3A3]
	BaseThreadInitThunk [0x00007FFBEE71257D+29]
	RtlUserThreadStart [0x00007FFBEF1EAA58+40]


In [23]:
data_dict

{'industry': 'Consumer Discretionary',
 'wr1': -0.001899418974503372,
 'wr2': -0.08783903595604621,
 'wr3': 0.004731496175130208,
 'wr4': 0.058891010294953236,
 'vol1': 150690000,
 'vol2': 135234000,
 'vol3': 189620000,
 'vol4': 162880000}

In [24]:
print(ticker, input_date)

amzn 2020-03-04 00:00:00


In [25]:
pd.to_datetime(input_date).to_period('M')

Period('2020-03', 'M')

In [26]:
monthly_cpi_df.loc[monthly_cpi_df['date'].dt.to_period('M') == pd.to_datetime(input_date).to_period('M'), 'value'].iloc[0]  

'258.115'

In [27]:
list_dict

[{'industry': 'Technology',
  'wr1': 0.010835549902080526,
  'wr2': 0.03504072501128531,
  'wr3': -0.08668396716704786,
  'wr4': 0.028513038591325562,
  'vol1': 2630100,
  'vol2': 1644500,
  'vol3': 1820700,
  'vol4': 1681100,
  'pe_ratio': -95.48,
  'debt_to_equity': 0.62,
  'quick_ratio': 1.02,
  'total_shareholder_return': -4.48,
  'profit_margin': -13.23,
  'free_cash_margin': 24.88,
  'volatility': 0.023469984128739643,
  'cpi': 254.202,
  'interest_rate': 2.4,
  'unemployment_rate': 3.8,
  'sma': 187.07999999999996,
  'rsi': 72.80701754385962,
  'ema': 187.09916666666666,
  'label': 1},
 {'industry': 'Industrials',
  'wr1': 0.02664328616717447,
  'wr2': 0.018568138818965955,
  'wr3': -0.0056807854479040375,
  'wr4': -0.008100008713847811,
  'vol1': 508200,
  'vol2': 773200,
  'vol3': 595700,
  'vol4': 331100,
  'pe_ratio': 14.1,
  'debt_to_equity': 9.76,
  'quick_ratio': 0.5,
  'total_shareholder_return': 0.05,
  'profit_margin': 5.63,
  'free_cash_margin': -35.82,
  'volatility'

### Export data into csv

In [28]:
# remember to export data as csv
data_df = pd.DataFrame(list_dict)
data_df.to_csv(f'data_JB{page}.csv', index=False)


In [29]:
data_df

,industry,wr1,wr2,wr3,wr4,vol1,vol2,vol3,vol4,pe_ratio,...,profit_margin,free_cash_margin,volatility,cpi,interest_rate,unemployment_rate,sma,rsi,ema,label
0,Technology,0.010836,0.035041,-0.086684,0.028513,2630100,1644500,1820700,1681100,-95.48,...,-13.23,24.88,0.023470,254.202,2.40,3.8,187.080000,72.807018,187.099167,1
1,Industrials,0.026643,0.018568,-0.005681,-0.008100,508200,773200,595700,331100,14.10,...,5.63,-35.82,0.025931,244.955,0.91,4.3,58.243333,100.000000,58.265000,0
2,Consumer Discretionary,-0.020641,0.049231,0.035032,0.053827,334900,389600,409100,592200,31.00,...,14.40,26.14,0.017016,296.171,2.33,3.7,174.780000,4.993959,174.667083,1
3,Consumer Discretionary,-0.006934,-0.073611,-0.007239,-0.017776,10935000,22703500,7615300,8039400,7.99,...,11.90,10.35,0.033077,257.346,1.82,3.6,53.013333,40.198511,53.056667,1
4,Utilities,0.021710,0.064121,0.071753,0.004059,305300,571100,942800,1120400,10.15,...,33.12,78.97,0.032350,281.148,0.08,4.0,23.166667,61.864407,23.097917,1
5,Technology,-0.012771,0.001944,0.005292,0.053702,52539000,26638300,32305500,26963100,22.68,...,21.57,24.68,0.019420,305.109,5.08,3.6,123.983333,30.908308,123.208750,1
6,Health Care,-0.034370,-0.031210,-0.003256,0.000724,6104000,5206000,5063200,4757700,27.78,...,18.36,20.62,0.020116,292.296,0.33,3.6,103.536667,9.706020,103.741667,1
7,Consumer Discretionary,0.036668,0.084841,-0.031665,0.077383,4090900,3819300,5152700,4381100,26.51,...,7.94,18.23,0.023663,256.394,0.05,13.2,222.006667,84.428692,223.055833,0
8,Finance,-0.001703,0.001591,-0.015403,0.009926,566400,668500,575800,1059000,21.61,...,27.26,-14.12,0.021972,271.696,0.10,5.9,865.840000,51.090180,865.239167,1
9,Consumer Discretionary,0.021720,-0.089594,0.026436,-0.009555,475700,699800,1430400,1023500,59.09,...,5.63,2.45,0.026877,258.678,1.58,3.5,151.613333,81.264637,151.658333,1


In [30]:
page = 1

In [31]:
print(ticker, input_date)

amzn 2020-03-04 00:00:00
